In [37]:
import redis
import pymysql
import sys

In [43]:
'''
1. 先到redis中查询
2. redis中没有，到Mysql中查询，并将数据缓存到redis(设置过期时间)
3. 再查询一次，此时就从redis中直接拿
4. 过期时间到，redis中缓存的数据没了，重复第二步
'''
#连接redis
r = redis.Redis(host='192.168.1.175',port=6379,db='0',encoding='utf8', decode_responses=True)

In [39]:
#连接mysql
def connect_to_mysql():
    # 設定資料庫連線資訊
    db_setting = {
        "host" : 'localhost',
        "port" : 3306,
        "user" : 'root',
        "passwd" : 'frank0813',
        "db" : "topic_104",
        "charset" : 'utf8mb4'
        }
# 建立連線
    conn = pymysql.connect(**db_setting)
    print('Successfully connected to DB : {} !'.format(db_setting["db"]))
    return conn

In [5]:
conn = connect_to_mysql()

Successfully connected to DB : topic_104 !


In [6]:
#创建pymysql游标对象
cursor = conn.cursor()

In [7]:
sql = '''SELECT a.Job_ID, a.Job_Name, a.Company, a. salary_clean, a. Job_Description, a.Other, b.concate_jieba    
        FROM job_104 as a
        join jobconcate_jieba as b
        on a.Job_ID = b.Job_ID '''

In [8]:
cursor.execute(sql)

132555

In [9]:
data = cursor.fetchall()

In [102]:
type(list(data[0])[0])

str

In [59]:
for i in data:
    r.sadd('Job',str(i))

In [97]:
for i in data:
    r.hmset('Job',['Job_ID':i[0],'Job_Name':i[1],'Company':i[2],'salary_clean':i[3],'Job_Description':i[4],'Other':i[5],'concate_jieba':i[6]])

SyntaxError: invalid syntax (<ipython-input-97-93ef63c17241>, line 2)

In [44]:
result = r.hgetall('100*')

In [45]:
result

{'Job_Name': '作業員(供膳)',
 'Job_Description': '旺季需配合加班',
 'salary_clean': '月薪,28000,30000',
 'Company': '總信食品有限公司',
 'concate_jieba': '健團保,食品,餐飲,作業員,包裝',
 'Other': '需能配合加班，享勞健團保食品餐飲相關科系為佳',
 'Job_ID': '10062'}

In [65]:
a = r.sscan('Job',0,'*友信*',140000)

In [81]:
b = list(a[1])

In [82]:
b[0]

"('6x13h', 'IoT物聯網安全銷售高級業務專員', '友信行股份有限公司', '月薪,35000,40000', '1.資安服務銷售，如：滲透測試、弱點掃描與社交工程演練2.資安產品銷售，如：物聯網設備、主機與端點防護系統3.電腦整合方案銷售4.專案客戶開發及維繫管理5.負責專案內、外部相關工作之溝通協調6.業績目標達成7.應收帳款催收', '-須具業務銷售三年以上經驗-資安專案業務經驗優先-具政府、金融或醫療電腦設備採購或資安設備採購標案經驗尤佳', '資安,服務,銷售,測試,掃描,社交,工程,演練,資安,產品,銷售,物聯網,設備,主機,端點,防護,系統,電腦,整合,方案,銷售,專案,客戶,開發,維繫,管理,專案,溝通,協調,業績,帳款,催收,業務,銷售,資安,專案,業務,政府,金融,醫療,電腦,設備,採購,資安,設備,採購,標案,業務,Excel,Outlook,PowerPoint,Word')"

In [96]:
b[0].split(',')[0].split(''/')

SyntaxError: EOL while scanning string literal (<ipython-input-96-b2216ab20fd1>, line 1)